
# WW Rand Mats

How the weightwatcher PL estimator performs on random matrices






In [41]:
# Suppress the powerlaw package warnings
# "powerlaw.py:700: RuntimeWarning: divide by zero encountered in true_divide"
# "powerlaw.py:700: RuntimeWarning: invalid value encountered in true_divide"
import warnings
warnings.simplefilter(action='ignore', category=RuntimeWarning)

In [1]:
import numpy as np
import pandas as pd

from tqdm import tqdm

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

2021-10-16T20:05:38-07:00

CPython 3.8.5
IPython 7.18.1

compiler   : Clang 10.0.0 
system     : Darwin
release    : 17.7.0
machine    : x86_64
processor  : i386
CPU cores  : 12
interpreter: 64bit


### Import WeightWatcher

set custom Logging at WARN Level

In [ ]:
import sys
if 'google.colab' in sys.modules:
    !pip install weightwatcher

In [2]:
import logging

import weightwatcher as ww
import torchvision.models as models

logger = logging.getLogger(ww.__name__)
logger.setLevel(logging.INFO)

ww.__version__

'0.5.5'

###  Create a Dummy Model

In [5]:
import weightwatcher as ww
import torchvision.models as model

model = models.vgg11(pretrained=True)
watcher = ww.WeightWatcher(model=model)
details = watcher.describe()

In [7]:
details['Q']=details.N/details.M

In [8]:
details

,layer_id,name,M,N,layer_type,num_evals,rf,Q
0,2,None,3.0,64.0,LAYER_TYPE.CONV2D,27.0,9.0,21.333333
1,5,None,64.0,128.0,LAYER_TYPE.CONV2D,576.0,9.0,2.000000
2,8,None,128.0,256.0,LAYER_TYPE.CONV2D,1152.0,9.0,2.000000
3,10,None,256.0,256.0,LAYER_TYPE.CONV2D,2304.0,9.0,1.000000
4,13,None,256.0,512.0,LAYER_TYPE.CONV2D,2304.0,9.0,2.000000
5,15,None,512.0,512.0,LAYER_TYPE.CONV2D,4608.0,9.0,1.000000
6,18,None,512.0,512.0,LAYER_TYPE.CONV2D,4608.0,9.0,1.000000
7,20,None,512.0,512.0,LAYER_TYPE.CONV2D,4608.0,9.0,1.000000
8,25,None,4096.0,25088.0,LAYER_TYPE.DENSE,4096.0,1.0,6.125000
9,28,None,4096.0,4096.0,LAYER_TYPE.DENSE,4096.0,1.0,1.000000


### Q ~= 1, 2, 4, 6, 21

In [22]:
import powerlaw
Qs , alphas = [], []
for layer_id in [28, 13, 31, 25, 2]:
    Q = details[details.layer_id == layer_id].Q.to_numpy()[0]
    
    esd = watcher.get_ESD(layer=layer_id, random=True)
    fit = powerlaw.Fit(esd, xmax=np.max(esd))
    alpha = fit.alpha
    print("Q={}  alpha={:0.2}".format(Q, alpha))
    Qs.append(Q)
    alphas.append(alpha)

Calculating best minimal value for power law fit


Q=1.0  alpha=8.6


Calculating best minimal value for power law fit


Q=2.0  alpha=8.4


Calculating best minimal value for power law fit
/Users/charleshmartin/anaconda3/envs/ww0.5/lib/python3.8/site-packages/powerlaw.py:1178: RuntimeWarning: overflow encountered in double_scalars
  return (self.alpha-1) * self.xmin**(self.alpha-1)
/Users/charleshmartin/anaconda3/envs/ww0.5/lib/python3.8/site-packages/powerlaw.py:825: RuntimeWarning: invalid value encountered in multiply
  likelihoods = f*C


Q=4.096  alpha=2.4e+01


Calculating best minimal value for power law fit


Q=6.125  alpha=1.4e+01


Calculating best minimal value for power law fit


Q=21.333333333333332  alpha=1.5e+01
